In [1]:
import os
os.chdir('/Users/markjos/projects/malachor5')
import sys
sys.path.append('scripts')
from longform import load_and_resample
import torchaudio
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


In [2]:
nchlt_filestem = 'data/nchlt/nchlt_'
nchlt_hf_dir = 'data/hf-datasets/nchlt_'
xslt_path = 'data/nchlt/tabulate_corpus.xslt'

split_paths = {}
for lang in ['eng', 'zul']:
    lang_filestem = nchlt_filestem + lang
    lang_hf_dir = nchlt_hf_dir + lang
    split_paths[lang] = {'data_dir': lang_filestem, 'hf_dir': lang_hf_dir}
    for split_short, split_long in {'trn': 'train', 'dev': 'validation', 'tst': 'test'}.items():
        split_fp = lang_filestem + '.' + split_short + '.lst'
        with open(split_fp) as f:
            split_records = f.readlines()
            split_records = [x.strip() for x in split_records]
        split_paths[lang][split_long] = split_records
        print(f'Found {len(split_records)} {lang} {split_long} records')

Found 71142 eng train records
Found 3038 eng validation records
Found 3232 eng test records
Found 38926 zul train records
Found 2945 zul validation records
Found 2802 zul test records


In [6]:
lang_dfs={}
for lang, lang_dict in [('eng', split_paths['eng'])]:#split_paths.items():
    df_list=[]
    ds_path = lang_dict['hf_dir']
    data_path = lang_dict['data_dir']
    os.makedirs(ds_path, exist_ok=True)
    for split in ['train', 'validation', 'test']:
        if split == 'test':
            split_abbrev = 'tst'
        else:
            split_abbrev = 'trn'
        os.makedirs(os.path.join(ds_path,split), exist_ok=True)
        print(f"Saving audio files for {split} split of {lang} dataset")
        transcription_df=pd.read_xml(
            os.path.join(data_path, 'transcriptions', f'nchlt_{lang}.{split_abbrev}.xml',),
            stylesheet=xslt_path,
        )
        transcription_df=transcription_df.set_index('file_name')
        for audio in tqdm(lang_dict[split]):
            subdir = audio.split('_')[2][:3]
            src_path = os.path.join(data_path,'audio',subdir, audio+'.wav')
            tgt_path = os.path.join(split,audio+'.wav')
            tgt_path_full = os.path.join(ds_path,tgt_path)
            if not os.path.exists(tgt_path_full):
                torchaudio.save(
                    tgt_path_full,
                    load_and_resample(src_path),
                    16_000
                )
        transcription_df=transcription_df.reset_index()
        transcription_df['split'] = split
        transcription_df['file_name']=transcription_df['split']+'/'+transcription_df['file_name'].apply(os.path.basename)
    df_list.append(transcription_df)
    df=pd.concat(df_list)
    df.to_csv(os.path.join(ds_path,'metadata.csv'),index=False)
    lang_dfs[lang]=df
    ds=load_dataset('audiofolder', data_dir=ds_path)
    ds.save_to_disk(lang_hf_dir.replace('hf-datasets', 'pyarrow-datasets'))
lang_dfs


Saving audio files for train split of eng dataset


100%|██████████| 71142/71142 [02:05<00:00, 565.54it/s]


Saving audio files for validation split of eng dataset


100%|██████████| 3038/3038 [00:05<00:00, 535.09it/s]


Saving audio files for test split of eng dataset


Resolving data files: 100%|██████████| 3232/3232 [00:00<00:00, 91719.10it/s]
Extracting data files: 0it [00:00, ?it/s]
Extracting data files: 0it [00:00, ?it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 0 examples [00:00, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset